In [ ]:


# the greek god of wealth (son of Iason and Demeter)
# dependencies

import os
import math
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# for loading data
from scipy.io import loadmat
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from random import randint
import matplot

import alpaca_trade_api as tradeapi

# api keys
api = tradeapi.REST(
        'AKUGANQEC0256T5OKJJA',
        'LNPrnn2jpq8HTRb86xv7jeEfV4qPxJbJz18IozgD',
        'https://api.alpaca.markets',
        api_version = 'v2'
    )


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device: ', device)

# nasdaq tickers
# can add from other exchanges
nasdaq = pd.read_csv('nasdaq.csv', sep=',', low_memory=False)
tickers = np.array(nasdaq['Symbol'])

In [ ]:

# getData
# String -> [int]
# The purpose of getData is to build the predictors to train our
# classifier. It will return the above data for the given ticker.
def getData(ticker):
    predictor = []
    barset = api.get_barset(ticker, 'day', limit=10)
    bars = barset[ticker]
    if(len(bars) != 10):
        return None
    # append the last ten closing prices as features
    for x in range(10):
        predictor.append(bars[x].c/bars[0].c)
    predictor = predictor
    # append the last ten volumes as features
    for y in range(10):
        predictor.append(bars[y].v/bars[0].v)
    #toReturn = np.array(predictor)
    return predictor

# call to get the volume data

#classifyData
# [float] -> int 
# classifyData is passed a set of features (a predictor) 
# and classifies the data point as profitable (purchase)
# or not profitable. In essence, we want to confirm that 
# drawbacks in price (if a closing price is less than the previous day)
# are increasingly high (so, the stock is still on a positive trajectory) <-- is the sample size too small?
# and that the volume trend confirms the price trend (volume is generally increasing?)

# the thing is, we don't want the neural network to learn
# our basic, scuffed up function
# we want it to identify patterns of its own
def classifyData(predictor):
    # we begin with the assumption of profitability
    profitable = True
    # move through the closing prices
    # of the stock

    # we are going to go with higher lows (in terms of drawbacks)
    
    # so first, we identify the first drawback in price (if it exists in the 10-day span)
    for x in range(10):
        if(x > 0):
            if(predictor[x] < predictor[x - 1]):
                low = predictor[x]
                # exit the loop
                break
    # if we make it through the loop without a drawback
    # then, profitable does not become False
    for x in range(10):
        if(x > 0):
            if(predictor[x] < predictor[x - 1]):
                if(predictor[x] < low):
                    profitable = False
                    break
                else:
                    # else, we keep moving through the list, 
                    # with the new drawback set as the low
                    low = predictor[x]
    # now, the volume requirements:
    # if it has made it past this point, then the price is on a bullish trend
    # volume has to "confirm" the trend
    # "higher highs" and "lower lows"
    p = 10
    while(p < 20):
        if(p > 10):
            if(predictor[p] < predictor[p - 1]):
                volLow = predictor[x]
                # exit the loop
                break
        p+=1
    u = 10
    while(u < 20):
        if(x > 0):
            if(predictor[x] < predictor[x - 1]):
                # it means the volume is not hitting a "higher high"
                if(predictor[x] < volLow):
                    profitable = False
                    break
                else:
                    # else, we keep moving through the list, 
                    # with the new drawback set as the low
                    volLow = predictor[x]
        u+=1
    return profitable


# build the training data for the classifier
def buildData(tickers):
    x_train = []
    y_train = []
    for ticker in tickers:
        if(getData(ticker) != None):
            toAdd = getData(ticker)
            x_train.append(toAdd)
            if(classifyData(toAdd)): 
                # will be a profitable stock
                y_train.append(1)
            else:
                # won't be a profitable stock
                y_train.append(0)
        else:
            # we do nothing
            continue
    return x_train, y_train


In [ ]:
x_train, y_train = buildData(tickers[:100])